<a href="https://colab.research.google.com/github/PosgradoMNA/actividades-de-aprendizaje-a01794067-janmenc/blob/main/ibm_modulo_4_data_analysis_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Maestría en Inteligencia Artificial Aplicada**

## **A01794067**
## **Jose Antonio Mendoza Castro**
## *Análisis de Datos Python*

## **Ciencia y analitica de datos**

## **Dra. Maria de la Paz Rico**

### **Desarrollo de modelos**

* Un modelo puede ser pensado como ecuación matemátca usada para predicir uno o varios valores dados.

* Relacionando una o más variables independientes a variables dependientes.

### Regresión lineal simple y múltiple

* La regresión lineal simple se refiere a la relación entre dos variables, una variable independiente y una dependiente. Está conformada por una constante y una pendiente. Cuando se sustituye un valor en la variable independiente obtenemos un valor estimado llamado predicción. A la diferencia entre nuestro valor estimado y el real le llamamos ruido. 

In [19]:
#en Python utilizamos la librería 'sklearn' para trabajar con regresiones lineales simples
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.linear_model import LinearRegression

In [2]:
#adquisicion de los datos
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'

In [3]:
#leemos el archivo omitiendo los 'headers' con el parametro 'header' igual a None
df = pd.read_csv(url, header = None)

In [4]:
headers = ["symboling","normalized-losses","make","fuel-type","aspiration", "num-of-doors","body-style",
         "drive-wheels","engine-location","wheel-base", "length","width","height","curb-weight","engine-type",
         "num-of-cylinders", "engine-size","fuel-system","bore","stroke","compression-ratio","horsepower",
         "peak-rpm","city-mpg","highway-mpg","price"]

In [5]:
#ponemos nombre a cada columna de acuerdo a la descripcion de los datos
df.columns = headers

In [6]:
#convertimos la variable precio a 'float'
#como tenemos datos faltantes identificados con el signo '?', primero obtenemos la media omitiendo esos datos
mean_price_tmp = df[df['price'] != '?']['price'].astype(int)
mean_price = np.mean(mean_price_tmp)
#una vez obtenida la media replazamos '?' por la media
df['price'] = df['price'].replace('?', mean_price) 
#finalmente convertimos toda la columna a 'float'
df['price'] = df['price'].astype(float) 

In [7]:
X = df[['highway-mpg']].copy()
y = df['price']

In [8]:
#creamos un objeto desde el 'constructor'
lm = LinearRegression()

In [9]:
lm.fit(X, y)

LinearRegression()

In [10]:
#con lo anterior podemos obtener una predicción
yhat = lm.predict(X)

In [11]:
#también podemos ocupar los siguientes atributos
#para el intercepto
lm.intercept_

37470.66014138312

In [12]:
#para el coeficiente o pendiente
lm.coef_

array([-789.02661986])

In [13]:
#también lo podemos hacerlo para un modelo de regresión lineal múltiple
#reemplazamos los valores faltantes en la variable 'horsepower'
mean_hp_tmp = df[df['horsepower'] != '?']['horsepower'].astype(int)
mean_hp = np.mean(mean_hp_tmp)

df['horsepower'] = df['horsepower'].replace('?', mean_price) 
df['horsepower'] = df['horsepower'].astype(int)

In [14]:
#tomaremos cuatro regresores para el modelo
z = df[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']].copy()

In [15]:
lm.fit(z, y)

LinearRegression()

In [16]:
yhat = lm.predict(z)

In [17]:
#también podemos ocupar los mismos atributos usados en la regresión simple
#para el intercepto
lm.intercept_

-6498.609211793937

In [18]:
#para el coeficiente o pendiente
lm.coef_

array([  -0.30635746,    3.53540121,  112.64181011, -115.54599117])

### Evaluación del modelo usando visualización